In [ ]:
import numpy as np
import os
import pandas as pd
import psycopg2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn.pipeline import Pipeline
from sklearn import linear_model
import matplotlib.pyplot as plt
from sklearn import preprocessing
from urllib.parse import urlparse
from urllib import parse
from lib.build_regressions_features_mixed import MixedFeatureData
from lib.explore_data import ExploratoryAnalysis
from lib.regressions import MixedClassificationModel

%matplotlib inline
FOLLOWER_URL = os.environ['FOLLOWER_URL']
                          
def make_connection():
    url_output = FOLLOWER_URL
    url = urlparse(url_output)
    conn = psycopg2.connect(
        database=url.path[1:],
        user=url.username,
        password=url.password,
        host=url.hostname
    )
    return conn

# START HERE
## The cell below contains all the mission specific data. Here you will specifity the mission id, the independent variables, the dependent variable, and the definitions of positive and negative outcome.

In [ ]:
#CHANGE THESE VALUES
mission_id = '24970'
#ONLY USE SINGLE QUESTIONS FOR DEPENDENT AND INDEPENDENT VARIABLES IN FORMAT 'PART_NUM-QUESTION_NUM'
continuous_independent_variables = ['2-7','2-9']
#NOTE MULTIPLE QUESTIONS NEED TO BE CATEGORICAL
categorical_independent_variables = ['2-3']
binary_independent_variables = ['2-8', '2-10', '2-16']
dependent_variable = '2-14'
negative_outcomes = ['Slightly satisfied', 'Neither satisfied nor dissatisfied',
                     'Slightly dissatisfied', 'Moderately dissatisfied', 'Extremely dissatisfied']
positive_outcomes = ['Extremely satisfied', 'Moderately satisfied']

# After entering the information above, go to Cell -> Run All to see your regression results

In [ ]:
cnc = make_connection()
questions = pd.read_sql_query("""with m_questions as (select id as question_id, label, type, position as question_position,
                                 part_id, structure from questions where type in ('SingleQuestion', 'MultipleQuestion', 'NumberQuestion')
                                 and mission_id = """+mission_id +"""),
                                 m_parts as (select id as part_id, position as part_position from parts where
                                 mission_id = """+mission_id+""")
                                 select question_id, label, type, question_position, m_parts.part_id, part_position,
                                 structure from m_questions join m_parts on m_questions.part_id = m_parts.part_id""", cnc)

question_list = '('+','.join([str(a) for a in questions['question_id'].tolist()])+')'
responses = pd.read_sql_query("""select id as response_id, snippet_id, question_id, answers from responses where 
                                 question_id in """+question_list, cnc)
response_data = pd.merge(responses, questions, on ='question_id')
cnc.close

In [ ]:
id_sample = response_data.groupby(['snippet_id']).count().reset_index().sample(frac=1)['snippet_id'].tolist()

In [ ]:
fd = MixedFeatureData(response_data[response_data['snippet_id'].isin(id_sample)].to_json(),
                 dependent_variable,
                 continuous_independent_variables,
                 binary_independent_variables,
                 categorical_independent_variables,
                 positive_outcomes,
                 negative_outcomes
                 )
eda = ExploratoryAnalysis(fd.encoded_features, fd.independent_variables,
                          fd.dependent_variable, fd.question_choices())
logistic_regression = MixedClassificationModel(fd)

## Correlations of Independent Variables
### After viewing these you may want to rethink your independent variable choices

In [ ]:
eda.correlation_matrix_plots()

In [ ]:
eda.correlation_matrix()

# Relative Variance

In [ ]:
explained_variances = eda.pca_explained_variances()
for i, q_id in enumerate(fd.independent_variables):
    print(q_id, round(explained_variances[i],2))

# Single Value Decomposition
## Data Projected onto a 2D space, and colored by Outcome

In [ ]:
eda.plot_outcome_clusters()

# Logistic Regression
## Histogram of Predicted Probability of Positive Outcome, colored by Actual Outcome


In [ ]:
logistic_regression.visualize_goodness()

#
## Results Summary

In [ ]:
logistic_regression.print_results()

# 
## Simulated Probability of Positive Outcome controlling for all but one variable

In [ ]:
for independent_variable in fd.continuous_independent_variables:
    if independent_variable not in fd.binary_independent_variables:
        logistic_regression.simulate_continuous_outcomes(independent_variable)

In [ ]:
for independent_variable in fd.independent_variables:
    if independent_variable in fd.binary_independent_variables or\
       independent_variable in fd.dummies:
        logistic_regression.simulate_binary_outcomes(independent_variable)